# Student Loan Risk with Deep Learning

In [24]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [25]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [26]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [27]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [28]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
y[:5]

array([0, 0, 0, 1, 0], dtype=int64)

In [29]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.copy()
X.drop("credit_ranking", axis=1, inplace=True)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [30]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [31]:
# Create a StandardScaler instance

scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [32]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

11

In [33]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = (number_input_features + 1) // 2

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) // 2

# Define the number of neurons in the output layer
n_outputs = 1

In [34]:
# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=n_outputs, activation = "sigmoid"))

In [35]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 72        
                                                                 
 dense_4 (Dense)             (None, 3)                 21        
                                                                 
 dense_5 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [36]:
# Compile the Sequential model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
# Fit the model using 50 epochs and the training data
nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 0s 755us/step - loss: 0.6857 - accuracy: 0.5897
Epoch 2/50
38/38 [==============================] - 0s 809us/step - loss: 0.6722 - accuracy: 0.6330
Epoch 3/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6637 - accuracy: 0.6722
Epoch 4/50
38/38 [==============================] - 0s 755us/step - loss: 0.6558 - accuracy: 0.6806
Epoch 5/50
38/38 [==============================] - 0s 809us/step - loss: 0.6470 - accuracy: 0.6939
Epoch 6/50
38/38 [==============================] - 0s 782us/step - loss: 0.6375 - accuracy: 0.7089
Epoch 7/50
38/38 [==============================] - 0s 782us/step - loss: 0.6291 - accuracy: 0.7198
Epoch 8/50
38/38 [==============================] - 0s 809us/step - loss: 0.6209 - accuracy: 0.7239
Epoch 9/50
38/38 [==============================] - 0s 889us/step - loss: 0.6143 - accuracy: 0.7239
Epoch 10/50
38/38 [==============================] - 0s 863us/step - loss: 0.6074 - accuracy: 0.7389
E

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [38]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
model_loss, model_accuracy

13/13 - 0s - loss: 0.5352 - accuracy: 0.7525 - 129ms/epoch - 10ms/step


(0.535210371017456, 0.7524999976158142)

### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [39]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
nn.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [40]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
student_loans_model = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [41]:
# Make predictions with the test data
predictions = student_loans_model.predict(X_test_scaled)

# Display a sample of the predictions
predictions

13/13 [==============================] - 0s 831us/step


array([[0.45743337],
       [0.42749435],
       [0.750168  ],
       [0.66174984],
       [0.750168  ],
       [0.5741174 ],
       [0.750168  ],
       [0.2578944 ],
       [0.6251825 ],
       [0.322058  ],
       [0.750168  ],
       [0.3378563 ],
       [0.5434498 ],
       [0.750168  ],
       [0.4652278 ],
       [0.3283985 ],
       [0.66554815],
       [0.42366225],
       [0.43321207],
       [0.50551355],
       [0.6898595 ],
       [0.6778947 ],
       [0.30527833],
       [0.750168  ],
       [0.49730965],
       [0.750168  ],
       [0.750168  ],
       [0.678883  ],
       [0.31055328],
       [0.46237904],
       [0.5519364 ],
       [0.750168  ],
       [0.25338393],
       [0.750168  ],
       [0.13721667],
       [0.5306455 ],
       [0.20448689],
       [0.36724466],
       [0.750168  ],
       [0.13775116],
       [0.750168  ],
       [0.06612382],
       [0.12216248],
       [0.74946284],
       [0.05629456],
       [0.5463746 ],
       [0.19705185],
       [0.295

In [42]:
# Save the predictions to a DataFrame and round the predictions to binary results
df = pd.DataFrame({"Predicted Values": predictions[:, 0]})
df["Predicted Values"] = df["Predicted Values"].apply(round)
df.head(30)

,Predicted Values
0,0
1,0
2,1
3,1
4,1
5,1
6,1
7,0
8,1
9,0


### Step 4: Display a classification report with the y test data and predictions

In [43]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, df["Predicted Values"]))

              precision    recall  f1-score   support

           0       0.73      0.74      0.74       188
           1       0.77      0.76      0.76       212

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.75      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

